##1. Загрузка пакетов - библиотек

In [34]:
# Импорт необходимых библиотек
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
import random

# Установка seed для воспроизводимости результатов
np.random.seed(42)
tf.random.set_seed(42)

##2. Загрузка датасета

In [35]:
dataset_root = Path("/content/iad-lab3/Vehicles")

# Проверка наличия датасета
if dataset_root.exists():
    print("Датасет уже найден по указанному пути.")
else:
    print("Датасет не найден. Выполняется загрузка с GitHub...")
    !git clone https://github.com/Daria-Chernykh/iad-lab3.git

# Получение списка классов (подкаталогов)
class_names = sorted(
    [item.name for item in dataset_root.iterdir() if item.is_dir()]
)

# Фиксация количества классов
num_classes = len(class_names)

print("\nНайденные классы:")
for idx, class_name in enumerate(class_names):
    print(f"{idx + 1}. {class_name}")

print(f"\nКоличество классов k = {num_classes}")

# Допустимые расширения изображений (в нижнем регистре)
image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".webp"}

# Подсчёт количества изображений в каждой папке класса
print("\nКоличество изображений по классам:")

total_images = 0

for class_name in class_names:
    class_path = dataset_root / class_name

    num_images = sum(
        1 for f in class_path.iterdir()
        if f.is_file() and f.suffix.lower() in image_extensions
    )

    total_images += num_images
    print(f"{class_name}: {num_images}")

print(f"\nОбщее количество изображений в датасете: {total_images}")

Датасет уже найден по указанному пути.

Найденные классы:
1. Auto Rickshaws
2. Bikes
3. Cars
4. Motorcycles
5. Planes
6. Ships
7. Trains

Количество классов k = 7

Количество изображений по классам:
Auto Rickshaws: 800
Bikes: 800
Cars: 790
Motorcycles: 800
Planes: 800
Ships: 800
Trains: 800

Общее количество изображений в датасете: 5590


В лабораторной работе используется датасет изображений транспортных средств (Vehicles). Набор данных включает 7 классов:
* Авторикши (Auto Rickshaws),
* Велосипеды (Bikes),
* Машины (Cars),
* Мотоциклы (Motorcycles),
* Самолеты (Planes),
* Корабли (Ships),
* Поезда (Trains).

Каждый класс представлен отдельной папкой с изображениями. Общее количество изображений в датасете составляет 5590. Число изображений в классах близко по величине (от 790 до 800 изображений на класс), что позволяет считать датасет практически сбалансированным и пригодным для решения задачи многоклассовой классификации с использованием сверточных нейронных сетей.

##3. Выбор и фиксация размерности изображений

In [36]:
# Фиксация размерности входных изображений
img_height = 224
img_width = 224
input_shape = (img_height, img_width, 3)

# Размер мини-пакета
batch_size = 32

##4. Формирование обучающей, валидационной и тестовой выборок

####4.1 Физическое разбиение датасета по папкам

In [37]:
source_root = Path("/content/iad-lab3/Vehicles")
target_root = Path("/content/Vehicles_split")

train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".webp"}

# Создание структуры каталогов
for split in ["train", "val", "test"]:
    for class_dir in source_root.iterdir():
        if class_dir.is_dir():
            (target_root / split / class_dir.name).mkdir(parents=True, exist_ok=True)

# Разбиение изображений по классам
for class_dir in source_root.iterdir():
    if not class_dir.is_dir():
        continue

    images = [
        img for img in class_dir.iterdir()
        if img.is_file() and img.suffix.lower() in image_extensions
    ]

    random.shuffle(images)

    n_total = len(images)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)

    train_images = images[:n_train]
    val_images = images[n_train:n_train + n_val]
    test_images = images[n_train + n_val:]

    for img in train_images:
        shutil.copy(img, target_root / "train" / class_dir.name / img.name)

    for img in val_images:
        shutil.copy(img, target_root / "val" / class_dir.name / img.name)

    for img in test_images:
        shutil.copy(img, target_root / "test" / class_dir.name / img.name)

print("Разбиение датасета на train / val / test завершено.")


Разбиение датасета на train / val / test завершено.


####4.2 Создание генераторов изображений

In [38]:
# Генератор с аугментацией — только для обучающей выборки
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

# Генераторы без аугментации — для валидации и теста
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    target_root / "train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

val_generator = val_test_datagen.flow_from_directory(
    target_root / "val",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = val_test_datagen.flow_from_directory(
    target_root / "test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


Found 3352 images belonging to 7 classes.
Found 1117 images belonging to 7 classes.
Found 1118 images belonging to 7 classes.


####4.3 Вывод количества изображений в каждой выборке

In [39]:
num_train = train_generator.samples
num_val = val_generator.samples
num_test = test_generator.samples

print("Количество изображений в выборках:")
print(f"Обучающая выборка: {num_train}")
print(f"Валидационная выборка: {num_val}")
print(f"Тестовая выборка: {num_test}")
print(f"Общее количество изображений: {num_train + num_val + num_test}")

Количество изображений в выборках:
Обучающая выборка: 3352
Валидационная выборка: 1117
Тестовая выборка: 1118
Общее количество изображений: 5587


Исходный датасет был физически разделён на обучающую, валидационную и тестовую выборки в соотношении 60% / 20% / 20% отдельно для каждого класса. Аугментация данных применялась только к обучающей выборке, что позволяет повысить обобщающую способность модели и избежать искажения оценок качества на валидационных и тестовых данных.

##5. Построение сверточной нейронной сети «с нуля» (Conv2D + MaxPooling + Dense)